In [7]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from pathlib import Path

In [8]:
PROJECT = "filmaholic-398017"
DATASET = "filmaholic"
cache_path = Path("./raw_data")

In [9]:
tables_to_query = ['links', 'movies', 'tags', 'ratings']
dfs = {}

In [10]:
for table in tables_to_query:
    
    file_cache_path = cache_path.joinpath(f"raw-{table}.csv")
    
    if file_cache_path.is_file():
        dfs[table] = pd.read_csv(file_cache_path, dtype='str')
        
        print(f"Table {table} loaded from cache")
    
    else:
        query = f"""
        SELECT * FROM `{PROJECT}.{DATASET}.raw-{table}`
        """
    
        client = bigquery.Client(project=PROJECT)
        query_job = client.query(query)
        result = query_job.result()
        dfs[table] = result.to_dataframe()
        
        print(f"Table {table} loaded from bigquery")
        
        if dfs[table].shape[0] > 1:
            dfs[table].to_csv(file_cache_path, index=False)
            
            print(f"Table {table} cached")

Table links loaded from cache
Table movies loaded from cache
Table tags loaded from cache
Table ratings loaded from cache


# Simple correlation

In [11]:
dfs['movies']

,movieId,title,genres
0,777,Pharaoh's Army (1995),War
1,1450,Prisoner of the Mountains (Kavkazsky plennik) ...,War
2,2669,Pork Chop Hill (1959),War
3,2670,Run Silent Run Deep (1958),War
4,3339,Cross of Iron (1977),War
...,...,...,...
62418,1907,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...
62419,56152,Enchanted (2007),Adventure|Animation|Children|Comedy|Fantasy|Mu...
62420,5018,Motorama (1991),Adventure|Comedy|Crime|Drama|Fantasy|Mystery|S...
62421,26093,"Wonderful World of the Brothers Grimm, The (1962)",Adventure|Animation|Children|Comedy|Drama|Fant...


In [12]:
merged = dfs['ratings'].merge(dfs['movies'], on='movieId', how='left')

In [13]:
pivoted = merged.pivot_table(index='userId', columns='title', values='rating')

/tmp/ipykernel_530230/627713098.py:1: PerformanceWarning: The following operation may generate 9583092278 cells in the resulting pandas object.
  pivoted = merged.pivot_table(index='userId', columns='title', values='rating')


: 

: 